In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio

In [2]:
from IPython.display import Audio
import numpy as np
import librosa

## Decompose Music

In [3]:
file_name = "/data/omkar/beat_detection/decomposition/Atif Aslam： Pehli Dafa Song (Video) ｜ Ileana D’Cruz ｜ Latest Hindi Song 2017 ｜ T-Series.wav"

In [4]:
song_name = file_name.split("/")[-1]
inst_name = f"{song_name.split('.wav')[0].split('.mp3')[0]}_(Instrumental)_model_mel_band_roformer_ep_3005_sdr_11.wav"
vocs_name = f"{song_name.split('.wav')[0].split('.mp3')[0]}_(Vocals)_model_mel_band_roformer_ep_3005_sdr_11.wav"

In [5]:
inst_name

'Atif Aslam： Pehli Dafa Song (Video) ｜ Ileana D’Cruz ｜ Latest Hindi Song 2017 ｜ T-Series_(Instrumental)_model_mel_band_roformer_ep_3005_sdr_11.wav'

In [107]:
from audio_separator.separator import Separator

# Initialize the Separator class (with optional configuration properties, below)
separator = Separator()

# Load a machine learning model (if unspecified, defaults to 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt')
separator.load_model()

# Perform the separation on specific audio files without reloading the model
output_files = separator.separate(file_name)

print(f"Separation complete! Output file(s): {' '.join(output_files)}")

2025-01-19 18:26:10,400 - INFO - separator - Separator version 0.28.5 instantiating with output_dir: None, output_format: WAV
2025-01-19 18:26:10,400 - INFO - separator - Output directory not specified. Using current working directory.
2025-01-19 18:26:10,401 - INFO - separator - Operating System: Linux #17~22.04.1-Ubuntu SMP Tue Sep  3 16:11:52 UTC 2024
2025-01-19 18:26:10,402 - INFO - separator - System: Linux Node: ml-platform-prod-dev-002 Release: 6.8.0-1015-gcp Machine: x86_64 Proc: x86_64
2025-01-19 18:26:10,403 - INFO - separator - Python Version: 3.10.16
2025-01-19 18:26:10,404 - INFO - separator - PyTorch Version: 2.4.1+cu121
2025-01-19 18:26:10,416 - INFO - separator - FFmpeg installed: ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
2025-01-19 18:26:10,418 - INFO - separator - ONNX Runtime GPU package installed with version: 1.20.1
2025-01-19 18:26:10,418 - INFO - separator - CUDA is available in Torch, setting Torch device to CUDA
2025-01-19 18:26:10,419 - 

Separation complete! Output file(s): Atif Aslam： Pehli Dafa Song (Video) ｜ Ileana D’Cruz ｜ Latest Hindi Song 2017 ｜ T-Series_(Instrumental)_model_mel_band_roformer_ep_3005_sdr_11.wav Atif Aslam： Pehli Dafa Song (Video) ｜ Ileana D’Cruz ｜ Latest Hindi Song 2017 ｜ T-Series_(Vocals)_model_mel_band_roformer_ep_3005_sdr_11.wav


In [10]:
vocal_file = vocs_name
instrumental_file = inst_name

### Transformations

In [11]:
audio, sr = librosa.load(instrumental_file)

In [12]:
# Log spectrogram:
spec = librosa.stft(audio)
log_spec = librosa.amplitude_to_db(np.abs(spec))

In [13]:
y_harmonic, y_percussive = librosa.effects.hpss(audio)
# plt.figure(figsize=(15, 5))
# librosa.display.waveshow(y_harmonic, sr=sr, alpha=0.25)
# librosa.display.waveshow(y_percussive, sr=sr, color='r', alpha=0.5)
# plt.title('Harmonic + Percussive')

In [14]:
audio.shape, spec.shape, log_spec.shape, y_harmonic.shape, y_percussive.shape

((6224385,), (1025, 12158), (1025, 12158), (6224385,), (6224385,))

In [15]:
from scipy.io.wavfile import write

write(f'{song_name[:-4]}"_percussive.wav', sr, y_percussive)
write(f'{song_name[:-4]}"_harmonic.wav', sr, y_harmonic)

In [16]:
from BeatNet.BeatNet import BeatNet

estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)

Output_harmonic = estimator.process(librosa.load(f'{song_name[:-4]}"_harmonic.wav')[0])
Output_percussion = estimator.process(librosa.load(f'{song_name[:-4]}"_percussive.wav')[0])

/home/omkarsb/anaconda3/envs/beats/lib/python3.10/site-packages/BeatNet/BeatNet.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(os

In [17]:
click_track_percussion = librosa.clicks(times=Output_percussion[:,0], sr=sr,
                             click_duration=0.25, length=len(audio))

In [18]:
click_track_harmonic = librosa.clicks(times=Output_harmonic[:,0], sr=sr,
                             click_duration=0.25, length=len(audio))

In [19]:
sr
window_duration = 0.3
window_length = int(window_duration * sr)
window_length

6615

In [20]:
total_length = click_track_percussion.shape[0]
n_iterations = int(total_length / window_length) + 1
n_iterations

941

In [21]:
indexes_harmonic = np.where((click_track_harmonic != 0) & (np.roll(click_track_harmonic, 1) == 0))[0]
indexes_percussion = np.where((click_track_percussion != 0) & (np.roll(click_track_percussion, 1) == 0))[0]

click_track_harmonic_oh = np.zeros_like(click_track_harmonic)
click_track_percussion_oh = np.zeros_like(click_track_percussion)

click_track_harmonic_oh[indexes_harmonic] = 1.0
click_track_percussion_oh[indexes_percussion] = 1.0

In [22]:
# final_track = np.array([], dtype=float)
# for idx in range(n_iterations) :
#     percussion_slice = click_track_percussion_oh[idx*window_length:min((idx+1)*window_length, total_length)]
#     harmonic_slice = click_track_harmonic_oh[idx*window_length:min((idx+1)*window_length, total_length)]
#     if harmonic_slice.sum() == 1 and percussion_slice.sum() == 1:
#         final_track = np.concatenate([final_track, percussion_slice], axis = 0)
#     else :
#         final_track = np.concatenate([final_track, np.zeros_like(percussion_slice)], axis = 0)

In [23]:
final_track = np.array([], dtype=float)
final_indexes = []
for idx in indexes_percussion :
    left_index = max(0, idx - int(window_length / 2))
    right_index= min(total_length, idx + int(window_length / 2))
    harmonic_slice = click_track_harmonic_oh[left_index:right_index+1]
    if harmonic_slice.sum() == 1 :
        final_indexes.append(idx)

final_track = np.zeros_like(click_track_harmonic_oh)
final_track[final_indexes] = 1

In [24]:
final_track = np.zeros_like(click_track_harmonic_oh)
final_track[final_indexes] = 1

In [25]:
len(final_indexes), sum(final_track)

(191, 191.0)

In [29]:
from collections import Counter

In [45]:
Counter([(final_indexes[i + 1] - final_indexes[i]) for i in range(len(final_indexes) - 1)])

Counter({23814: 69,
         23373: 63,
         22932: 19,
         24255: 16,
         23815: 7,
         23372: 4,
         23813: 4,
         23374: 2,
         1628613: 1,
         22490: 1,
         24256: 1,
         24254: 1,
         24696: 1,
         21609: 1})

In [32]:
import numpy as np
from collections import Counter

def correct_beat_pattern(beats, tolerance=0.2):
    """
    Corrects a binary beat pattern by identifying the true interval and phase.
    
    Args:
        beats: List[int] - Binary array where 1 represents a beat
        tolerance: float - Relative tolerance for considering intervals as equal
        
    Returns:
        List[int] - Corrected beat pattern
    """
    # Find positions of all beats
    beat_positions = np.where(beats)[0]
    
    if len(beat_positions) < 2:
        return beats
    
    # Calculate intervals between consecutive beats
    intervals = np.diff(beat_positions)
    
    # Group similar intervals to find the most common one
    rounded_intervals = [round(x) for x in intervals]
    interval_counts = Counter(rounded_intervals)
    true_interval = interval_counts.most_common(1)[0][0]
    
    # Create corrected beat pattern
    corrected = np.zeros_like(beats)
    
    # Find the optimal starting position by trying different phases
    best_score = -1
    best_offset = 0
    
    # Try different starting positions within one interval
    for offset in range(true_interval):
        score = 0
        test_positions = np.arange(offset, len(beats), true_interval)
        # Count how many original beats align with this phase
        for pos in test_positions:
            if pos < len(beats) and beats[int(pos)] == 1:
                score += 1
        
        if score > best_score:
            best_score = score
            best_offset = offset
    
    # Place beats at regular intervals from the best starting position
    beat_positions = np.arange(best_offset, len(beats), true_interval)
    for pos in beat_positions:
        corrected[int(pos)] = 1
    
    return corrected.tolist()

In [ ]:
# Example usage and testing
# Test case 1: Pattern starting with silence
test1 = [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
print("Test 1 - Pattern starting with silence:")
print("Original:", test1)
print("Corrected:", correct_beat_pattern(test1))

# Test case 2: Pattern with noise and arbitrary phase
test2 = [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0]
print("\nTest 2 - Pattern with noise:")
print("Original:", test2)
print("Corrected:", correct_beat_pattern(test2))

# Test case 3: Your original example
test3 = [0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
print("\nTest 3 - Original example:")
print("Original:", test3)
print("Corrected:", correct_beat_pattern(test3))

In [36]:
cfinal_track = correct_beat_pattern(final_track)

In [39]:
np.array(cfinal_track).sum()

262.0

In [ ]:
Counter([(cfinal_track[i + 1] - cfinal_track[i]) / sr for i in range(len(final_indexes) - 1)])

In [42]:
beat_positions = np.where(cfinal_track)[0]

# Calculate intervals between consecutive beats
intervals = np.diff(beat_positions)

In [44]:
Counter(intervals)

Counter({23814: 261})